<a href="https://colab.research.google.com/github/martinlee192/AI-basic/blob/master/ai_basic_mission5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1. 가장 먼저 학습 데이터를 준비해보도록 하겠습니다. MNIST 데이터셋을 직접 Load해 봅시다.

데이터셋을 로드하고 DataLoader를 구현해보세요.

DataLoader를 이용해 MNIST 데이터셋을 로드해봅시다.

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

# data loader
training_epochs = 15
batch_size = 100
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)


Q2.데이터가 준비가 되었다면, 이제 그 데이터를 학습할 모델을 구현할 차례입니다.

그 후 모델 안의 가중치를 초기화시켜보세요.

입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.

MNIST 입력의 크기는 28x28입니다.

여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰보세요.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias=True).to(device)
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 0.6211,  0.0524,  2.1839,  ..., -0.9368,  0.0996,  1.0293],
        [-0.4191, -0.5264,  1.5265,  ..., -0.1950,  0.6707,  0.5747],
        [-1.4621, -0.5727,  0.6351,  ..., -0.4586, -0.1824, -0.6445],
        ...,
        [ 0.3259, -0.4568,  1.4383,  ...,  0.4484, -0.0428,  0.9264],
        [-0.4592,  0.4501, -1.7533,  ...,  0.0556,  1.8174,  0.9317],
        [ 0.4133,  0.5796,  1.4404,  ..., -0.6304,  0.6587, -0.4735]],
       requires_grad=True)

Q3. 위에서 구현한 모델을 학습시키기 위해서는 loss 함수와 opitmizer가 필요합니다.

아래 제시된 loss 함수와 optimizer를 구현해보세요.

Loss 함수와 optimizer는 모델 안의 가중치를 업데이트 할 때 사용됩니다.

옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [ ]:
# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

Q4. 3번 문제까지 해결하셨다면, 이제 학습을 위한 준비는 거의 끝났다고 볼 수 있습니다.

위 구현 함수들을 이용해 학습 Loop를 구현해보세요.

위에서 구현한 모델, optimzer, loss fn 등을 이용해 학습을 구현해주세요.

In [ ]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad() # optimizer zero grad 구현
        loss.backward() # loss backward 구현
        optimizer.step() # optimizer step 구현

        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

Epoch [1/15], Step [100/600], Loss: 3.4981, Accuracy: 50.00%
Epoch [1/15], Step [200/600], Loss: 2.2592, Accuracy: 65.00%
Epoch [1/15], Step [300/600], Loss: 1.8708, Accuracy: 64.00%
Epoch [1/15], Step [400/600], Loss: 1.8271, Accuracy: 66.00%
Epoch [1/15], Step [500/600], Loss: 1.1273, Accuracy: 76.00%
Epoch [1/15], Step [600/600], Loss: 0.7022, Accuracy: 82.00%
Epoch [2/15], Step [100/600], Loss: 1.4302, Accuracy: 76.00%
Epoch [2/15], Step [200/600], Loss: 1.6130, Accuracy: 66.00%
Epoch [2/15], Step [300/600], Loss: 0.8090, Accuracy: 85.00%
Epoch [2/15], Step [400/600], Loss: 1.1238, Accuracy: 80.00%
Epoch [2/15], Step [500/600], Loss: 1.2524, Accuracy: 81.00%
Epoch [2/15], Step [600/600], Loss: 1.1085, Accuracy: 77.00%
Epoch [3/15], Step [100/600], Loss: 1.1797, Accuracy: 83.00%
Epoch [3/15], Step [200/600], Loss: 1.0209, Accuracy: 83.00%
Epoch [3/15], Step [300/600], Loss: 0.7149, Accuracy: 86.00%
Epoch [3/15], Step [400/600], Loss: 0.8910, Accuracy: 88.00%
Epoch [3/15], Step [500/

Q5. 학습이 완료되면, 모델이 잘 동작하는지 테스트가 필요합니다.

데이터로드 파트에서 준비했던 테스트 데이터를 이용해 테스트를 진행해봅시다.

아래 테스트 코드를 완성해보세요.

In [ ]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        output = linear(imgs)

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()
    
    print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images: 10.24%
